In [1]:
import torch
import time
from torch import mps

# compare performance between sort and topk of pytorch

data = torch.randn(20000, dtype=torch.float32, device=torch.device("mps:0"))
num_topk = 100


def topk1(num_topk):
    return data.topk(num_topk, sorted=False)


def topk2(num_topk):
    sort, idx = data.sort(descending=True)
    return sort[:num_topk], idx[:num_topk]


def benchmark(function, iter, warmup):
    # warmup step
    for k in range(warmup):
        function

    # implementation
    start = time.perf_counter()
    for k in range(iter):
        function
    torch.mps.synchronize()

    return time.perf_counter() - start


print("topk: ", benchmark(topk1(num_topk), 100, 3))
print("sort: ", benchmark(topk2(num_topk), 100, 3))

topk:  0.003571375011233613
sort:  0.0019330840150360018
